In [2]:
from typing import Any, List, Optional, Tuple


class DWayHeap(object):
    def __init__(self, elements: List[Any] = [], priorities: List[float] = [], branching_factor: int = 2) -> None:
        """Constructor

        Args:
            elements: Los elementos para inicializar el heap.
            priorities: Las prioridades de los elementos anteriores. Deben tener la misma longitud que `elements`.
            branching_factor: El número (máximo) de hijos por nodo en el heap. Debe ser al menos 2.
        """
        if len(elements) != len(priorities):
            raise ValueError(f'La longitud de la lista de elementos ({len(elements)})'
                             f' debe coincidir con la longitud de la lista de prioridades ({len(priorities)}).')
        if branching_factor < 2:
            raise ValueError(f'El factor de ramificación ({branching_factor}) debe ser mayor que 1.')
        self._pairs: List[Tuple[float, Any]] = []
        self.D = branching_factor

        if len(elements) > 0:
            self._heapify(elements, priorities)

    def __sizeof__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self)

    def __len__(self) -> int:
        """Tamaño del heap.

        Returns: El número de elementos en el heap.
        """
        return len(self._pairs)

    def _validate(self) -> bool:
        """Verifica que se cumplan las tres invariantes del heap:
        1. Cada nodo tiene como máximo `D` hijos. (Garantizado por la construcción)
        2. El árbol del heap es completo y alineado a la izquierda. (También garantizado por la construcción)
        3. Cada nodo contiene la mayor prioridad en el subárbol con raíz en ese nodo.

        Returns: True si se cumplen todas las invariantes del heap.
        """
        current_index = 0
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            current_priority: float = self._pairs[current_index][0]
            first_child = self._first_child_index(current_index)
            last_child_guard = min(first_child + self.D, len(self))
            for child_index in range(first_child, last_child_guard):
                if current_priority < self._pairs[child_index][0]:
                    return False
            current_index += 1
        return True

    def _push_down(self, index: int) -> None:
        """Empuja hacia abajo la raíz de un sub-heap hacia sus hojas para restablecer las invariantes del heap.
        Si alguno de los hijos del elemento tiene mayor prioridad, se intercambia el elemento actual
        con su hijo de mayor prioridad C, y se verifica recursivamente el sub-heap que estaba previamente
        enraizado en ese C.

        Args:
            index: El índice de la raíz del sub-heap.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        current_index = index
        first_leaf = self.first_leaf_index()
        while current_index < first_leaf:
            child_index = self._highest_priority_child_index(current_index)
            assert (child_index is not None)
            if self._pairs[child_index][0] > input_priority:
                self._pairs[current_index] = self._pairs[child_index]
                current_index = child_index
            else:
                break

        self._pairs[current_index] = input_pair

    def _bubble_up(self, index: int) -> None:
        """Eleva un elemento hacia la raíz para restablecer las invariantes del heap.
        Si el padre P de un elemento tiene menor prioridad, se intercambian el elemento actual y su padre,
        y se verifica recursivamente la posición que tenía P anteriormente.

        Args:
            index: El índice del elemento a elevar.
        """
        assert (0 <= index < len(self._pairs))
        input_pair = self._pairs[index]
        input_priority = input_pair[0]
        while index > 0:
            parent_index = self._parent_index(index)
            parent = self._pairs[parent_index]

            if input_priority > parent[0]:
                self._pairs[index] = parent
                index = parent_index
            else:
                break

        self._pairs[index] = input_pair

    def _first_child_index(self, index) -> int:
        """Calcula el índice del primer hijo de un nodo en el heap.

        Args:
            index: El índice del nodo actual, del cual se buscan los índices de sus hijos.

        Returns: El índice del hijo más a la izquierda del nodo actual del heap.
        """
        return index * self.D + 1

    def _parent_index(self, index) -> int:
        """Calcula el índice del padre de un nodo en el heap.

        Args:
            index: El índice del nodo actual, del cual se busca el índice de su padre.

        Returns: El índice del padre del nodo actual del heap.
        """
        return (index - 1) // self.D

    def _highest_priority_child_index(self, index) -> Optional[int]:
        """Encuentra, entre los hijos de un nodo del heap, el hijo con la mayor prioridad.
        En caso de que varios hijos tengan la misma prioridad, se devuelve el más a la izquierda.

        Args:
            index: El índice del nodo del heap cuyos hijos se examinan.

        Returns: El índice del hijo con mayor prioridad del nodo actual del heap, o None si
                 el nodo actual no tiene hijos.
        """
        first_index = self._first_child_index(index)
        size = len(self)
        last_index = min(first_index + self.D, size)

        if first_index >= size:
            return None

        highest_priority = -float('inf')
        index = first_index
        for i in range(first_index, last_index):
            if self._pairs[i][0] > highest_priority:
                highest_priority = self._pairs[i][0]
                index = i

        return index

    def first_leaf_index(self):
        return (len(self) - 2) // self.D + 1

    def _heapify(self, elements: List[Any], priorities: List[float]) -> None:
        """Inicializa el heap con una lista de elementos y prioridades.

        Args:
            elements: La lista de elementos a agregar al heap.
            priorities: Las prioridades correspondientes a esos elementos (en el mismo orden).
        """
        assert (len(elements) == len(priorities))
        self._pairs = list(zip(priorities, elements))
        last_inner_node_index = self.first_leaf_index() - 1
        for index in range(last_inner_node_index, -1, -1):
            self._push_down(index)

    def is_empty(self) -> bool:
        """Verifica si el heap está vacío.

        Returns: True si el heap está vacío.
        """
        return len(self) == 0

    def top(self) -> Any:
        """Elimina y devuelve el elemento con mayor prioridad en el heap.
        Si el heap está vacío, lanza una `RuntimeError`.

        Returns: El elemento con mayor prioridad en el heap.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método top en un heap vacío.')
        if len(self) == 1:
            element = self._pairs.pop()[1]
        else:
            element = self._pairs[0][1]
            self._pairs[0] = self._pairs.pop()
            self._push_down(0)

        return element

    def peek(self) -> Any:
        """Devuelve, SIN eliminarlo, el elemento con mayor prioridad en el heap.
        Si el heap está vacío, lanza una `RuntimeError`.

        Returns: El elemento con mayor prioridad en el heap.
        """
        if self.is_empty():
            raise RuntimeError('Se llamó al método peek en un heap vacío.')
        return self._pairs[0][1]

    def insert(self, element: Any, priority: float) -> None:
        """Agrega un nuevo par elemento/prioridad al heap

        Args:
            element: El nuevo elemento a agregar.
            priority: La prioridad asociada al nuevo elemento.
        """
        self._pairs.append((priority, element))
        self._bubble_up(len(self._pairs) - 1)


### Pruebas 

El código define un conjunto de pruebas unitarias para la clase `DWayHeap`, que implementa un heap d-ario. Utiliza las librerías unittest y random para generar datos de prueba. Se ejecutan todas las pruebas con un runner, comprobando el correcto comportamiento de la estructura de datos en cada caso exitosamente.

In [ ]:
import unittest
import random

# Factores de ramificación a probar
BRANCHING_FACTORS_TO_TEST = [2, 3, 4, 5, 6]

# Se asume que la clase DWayHeap ya está definida en el entorno.

class HeapTest(unittest.TestCase):
    def test_init(self):
        # Prueba de inicialización: se crea un heap vacío con factor de ramificación 2.
        heap = DWayHeap(branching_factor=2)
        self.assertEqual(0, len(heap))

        # Prueba con valores inválidos para el factor de ramificación (debe ser mayor que 1).
        for b in [1, 0, -1]:
            with self.assertRaises(ValueError) as context:
                DWayHeap(branching_factor=b)
            self.assertTrue(f'El factor de ramificación ({b}) debe ser mayor que 1.' in str(context.exception))

        # Prueba de inconsistencia entre la longitud de la lista de elementos y la lista de prioridades.
        with self.assertRaises(ValueError) as context:
            DWayHeap(priorities=[1.0])
        error_str = 'La longitud de la lista de elementos (0) debe coincidir con la longitud de la lista de prioridades (1).'
        self.assertTrue(error_str in str(context.exception))

        # Prueba de inicialización con elementos y prioridades válidos.
        heap = DWayHeap(elements=['A', 'B', 'C', 'D'], priorities=[0.1, -0.1, 1., -2.], branching_factor=2)
        self.assertEqual(4, len(heap))
        self.assertTrue(heap._validate())

    def test_heapify(self):
        # Prueba de la función heapify para distintos factores de ramificación.
        for b in BRANCHING_FACTORS_TO_TEST:
            # Se genera un tamaño aleatorio para el heap.
            size = 4 + random.randint(0, 20)
            # Se generan elementos (letras) y prioridades aleatorias.
            elements = [chr(i) for i in range(ord('A'), ord('A') + size)]
            priorities = [random.random() for _ in range(size)]
            heap = DWayHeap(elements=elements, priorities=priorities, branching_factor=b)
            self.assertEqual(size, len(heap))
            self.assertTrue(heap._validate())

    def test_clear(self):
        # Prueba del comportamiento en heaps vacíos y tras insertar elementos.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Se verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento con una prioridad muy baja.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.peek())

            # Inserta varios elementos con distintas prioridades.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.peek())

    def test_insert_top(self):
        # Prueba de la inserción y extracción (top) de elementos en el heap.
        for b in BRANCHING_FACTORS_TO_TEST:
            heap = DWayHeap(branching_factor=b)
            # Verifica que llamar a peek en un heap vacío arroja RuntimeError.
            with self.assertRaises(RuntimeError) as context:
                heap.peek()
            self.assertTrue('Se llamó al método peek en un heap vacío.' in str(context.exception))

            # Inserta el primer elemento y lo extrae utilizando top.
            heap.insert('First', -1e14)
            self.assertEqual('First', heap.top())
            self.assertTrue(heap.is_empty())

            # Inserta varios elementos y verifica que se extrae el de mayor prioridad.
            heap.insert("b", 0)
            heap.insert("c", 0.99)
            heap.insert("second", 1)
            heap.insert("a", -11)
            self.assertEqual('second', heap.top())
            self.assertEqual(3, len(heap))

            # Inserta 10 elementos adicionales con prioridades aleatorias.
            for i in range(10):
                heap.insert(str(i), random.random())

            # Extrae elementos hasta que el heap esté vacío, verificando la estructura en cada extracción.
            while not heap.is_empty():
                self.assertTrue(heap._validate())
                heap.top()

# Ejecuta las pruebas
suite = unittest.TestLoader().loadTestsFromTestCase(HeapTest)
unittest.TextTestRunner(verbosity=2).run(suite)


### Ejemplos

#### Ejemplo 1: Ordenación descendente usando el heap

Se insertan números aleatorios en el heap (usando su propio valor como prioridad) y luego se extraen en orden descendente (ya que se trata de un max-heap).

In [3]:
import random

# Genera una lista de 20 números aleatorios
numbers = [random.randint(1, 100) for _ in range(20)]
print("Lista original:", numbers)

# Crea una instancia de DWayHeap (factor de ramificación por defecto 2)
heap = DWayHeap()

# Inserta cada número en el heap; la prioridad es el propio número.
for num in numbers:
    heap.insert(num, num)

# Extrae los números del heap (se obtienen en orden descendente)
sorted_numbers = []
while not heap.is_empty():
    sorted_numbers.append(heap.top())

print("Ordenados de mayor a menor:", sorted_numbers)

Lista original: [12, 8, 98, 54, 63, 29, 68, 51, 5, 23, 55, 9, 6, 7, 21, 72, 25, 8, 50, 6]
Ordenados de mayor a menor: [98, 72, 68, 63, 55, 54, 51, 50, 29, 25, 23, 21, 12, 9, 8, 8, 7, 6, 6, 5]


#### Ejemplo 2: Cola de prioridades para gestionar tareas

Simula una cola de tareas en la que cada tarea tiene una prioridad. Se procesan las tareas en orden de mayor prioridad.


In [ ]:
# Definición de tareas: (nombre de la tarea, prioridad)
tasks = [
    ("Tarea A", 2),
    ("Tarea B", 5),
    ("Tarea C", 1),
    ("Tarea D", 4),
    ("Tarea E", 3),
]

# Se crea un heap con un factor de ramificación de 3
task_heap = DWayHeap(branching_factor=3)

# Inserta las tareas en el heap (prioridad mayor indica mayor importancia)
for task, priority in tasks:
    task_heap.insert(task, priority)

print("Procesando tareas por prioridad:")
while not task_heap.is_empty():
    print("Procesando:", task_heap.top())

#### Ejemplo 3: Benchmarking con distintos factores de ramificación

Mide el tiempo que tarda en insertar y extraer 10,000 elementos utilizando diferentes factores de ramificación.

In [ ]:
import time

num_elements = 10000
results = {}

for b in [2, 3, 4, 5, 6]:
    # Genera números aleatorios
    nums = [random.randint(1, 100000) for _ in range(num_elements)]
    
    # Inicializa el heap con el factor de ramificación b
    heap = DWayHeap(branching_factor=b)
    
    start_time = time.time()
    # Inserta todos los números en el heap
    for n in nums:
        heap.insert(n, n)
    
    # Extrae todos los números
    while not heap.is_empty():
        heap.top()
    
    elapsed = time.time() - start_time
    results[b] = elapsed

print("Resultados del benchmark (factor de ramificación: tiempo en segundos):")
for b, t in results.items():
    print(f"{b}: {t:.4f} segundos")

#### Ejemplo 4: Validación de invariantes durante operaciones aleatorias

Inserta y extrae elementos de forma aleatoria, verificando periódicamente que las invariantes del heap se mantienen correctamente.

In [ ]:
heap = DWayHeap(branching_factor=4)
# Inserta 50 elementos aleatorios y valida la estructura cada 10 inserciones.
for i in range(50):
    heap.insert(random.randint(1, 100), random.random())
    if i % 10 == 0:
        print(f"Invariante válida tras la inserción {i}:", heap._validate())

# Extrae todos los elementos, comprobando la validez del heap en cada paso.
while not heap.is_empty():
    heap.top()
    print("Estructura válida durante extracción:", heap._validate())

#### Ejemplo 5: Simulación de actualización de prioridad  

Aunque la clase no implementa un método update, se puede simular actualizando directamente la prioridad de un elemento y reequilibrando el heap. En este ejemplo se define una función auxiliar para ello.

In [ ]:
def update_priority(heap: DWayHeap, element: Any, new_priority: float) -> None:
    # Buscar el índice del elemento en el heap.
    index = None
    for i, (priority, elem) in enumerate(heap._pairs):
        if elem == element:
            index = i
            break
    if index is None:
        print("Elemento no encontrado")
        return
    old_priority = heap._pairs[index][0]
    heap._pairs[index] = (new_priority, element)
    # Si la nueva prioridad es mayor, se eleva; de lo contrario, se empuja hacia abajo.
    if new_priority > old_priority:
        heap._bubble_up(index)
    else:
        heap._push_down(index)

# Ejemplo de uso:
heap = DWayHeap()
heap.insert("Tarea importante", 10)
heap.insert("Tarea media", 5)
heap.insert("Tarea baja", 1)

print("Prioridad superior antes de actualización:", heap.peek())
update_priority(heap, "Tarea baja", 12)
print("Prioridad superior después de actualizar 'Tarea baja':", heap.peek())

#### Ejemplo 6: Manejo de duplicados  
Se insertan elementos con la misma prioridad. La implementación elige el hijo más a la izquierda en caso de empate, lo que permite obtener un orden predecible.


In [ ]:
heap = DWayHeap()
heap.insert("A", 10)
heap.insert("B", 10)
heap.insert("C", 10)

print("Extracción de elementos con prioridad duplicada:")
while not heap.is_empty():
    print(heap.top())

#### Ejemplo 7: Uso de heapify con listas predefinidas  
Se construye un heap a partir de listas de elementos y prioridades. Luego se extraen los elementos en orden de mayor prioridad.

In [ ]:
elements = ['Z', 'Y', 'X', 'W']
priorities = [1, 4, 2, 3]
heap = DWayHeap(elements=elements, priorities=priorities, branching_factor=3)

print("Extracción en orden de prioridad (heapify):")
while not heap.is_empty():
    print(heap.top())

#### Ejemplo 8: Uso con objetos complejos  

Se utiliza el heap para gestionar objetos complejos (por ejemplo, diccionarios) donde la prioridad se define a partir de un atributo (como la edad).


In [ ]:
people = [
    {"name": "Ana", "age": 28},
    {"name": "Luis", "age": 35},
    {"name": "María", "age": 22},
    {"name": "Carlos", "age": 30},
]

# Se crea el heap utilizando la edad como prioridad.
heap = DWayHeap()
for person in people:
    heap.insert(person, person["age"])

print("Personas ordenadas por edad (mayor primero):")
while not heap.is_empty():
    person = heap.top()
    print(f"{person['name']} - {person['age']} años")

### Ejercicios

1. **Implementación de Decrease-Key y Increase-Key**  

Diseña y documenta cómo extender la clase DWayHeap para soportar operaciones de actualización de prioridad (tanto para reducir como para aumentar la clave) de un elemento existente. Deberás mantener las invariantes del heap y analizar la complejidad en cada caso, considerando la necesidad de reubicar el elemento hacia arriba o hacia abajo según corresponda.

2. **Fusión de heaps (Meld o Merge)**  

Propón un algoritmo para fusionar dos instancias de DWayHeap en una única estructura sin reconstruir el heap desde cero. Analiza y justifica el orden de complejidad del algoritmo y discute las implicaciones en la preservación de las invariantes de la estructura.

3. **Generalización a comparadores arbitrarios**  

Rediseña la estructura para que, en lugar de depender únicamente de prioridades numéricas, acepte una función de comparación personalizada. Debes garantizar que las operaciones de inserción, extracción y validación del heap se adapten correctamente a cualquier criterio de ordenación definido por el usuario, manteniendo el comportamiento esperado.

4. **Soporte para concurrencia y acceso multihilo**  

Plantea un diseño para adaptar el DWayHeap a entornos concurrentes, donde múltiples hilos puedan realizar operaciones de inserción, extracción y actualización simultáneamente. Define un esquema de sincronización (por ejemplo, mediante bloqueos o estrategias lock-free) que asegure la integridad y consistencia de la estructura, y discute posibles cuellos de botella.

5. **Análisis comparativo de rendimiento**  

Diseña una serie de experimentos teóricos y empíricos para comparar el rendimiento del DWayHeap frente a un heap binario tradicional y, de ser posible, frente a otras estructuras de colas de prioridad como el Heap de Fibonacci. Investiga cómo varían las operaciones clave (inserción, extracción, actualización) en función del factor de ramificación y el tamaño del heap, justificando tus hallazgos.

6. **Construcción en tiempo lineal a partir de un arreglo desordenado**  

Demuestra teóricamente y luego implementa (a nivel de enunciado) un algoritmo que convierta un arreglo de elementos y sus prioridades en un DWayHeap en tiempo O(n). Justifica matemáticamente cada paso del proceso y explica cómo se garantiza la correctitud de las invariantes del heap.

7. **Aplicación en un scheduler de tareas en tiempo real**  

Imagina la adaptación del DWayHeap para funcionar como núcleo de un planificador (scheduler) de tareas en sistemas en tiempo real. Define cómo se manejarán las prioridades dinámicas, la reprogramación de tareas y la eliminación de tareas caducadas, así como las implicaciones de rendimiento y latencia en entornos críticos.

8. **Estabilidad del heap en caso de prioridades iguales**  

Analiza el comportamiento del DWayHeap cuando se insertan múltiples elementos con la misma prioridad. Plantea modificaciones para garantizar la estabilidad (mantener el orden de inserción) y evalúa el impacto de estas modificaciones en la complejidad de las operaciones.


9. **Adaptación a búsqueda heurística (A\* y variantes)**  

Diseña cómo modificar el DWayHeap para que sirva como cola de prioridades en algoritmos de búsqueda heurística, considerando la posibilidad de actualizar dinámicamente las prioridades conforme cambian las estimaciones de costo. Analiza el impacto del factor de ramificación en la eficiencia del algoritmo.

10. **Persistencia y serialización**  

Plantea un mecanismo que permita serializar y deserializar el estado completo del DWayHeap en disco. Debes garantizar que la estructura se pueda almacenar y recuperar sin pérdida de información, considerando escenarios con grandes volúmenes de datos y la eficiencia de la operación.

11. **Heaps externos para grandes volúmenes de datos**  

Investiga y diseña una variante del DWayHeap orientada a trabajar con datos que no caben en memoria principal. Especifica estrategias de paginación, caché y acceso a disco, y discute las implicaciones de latencia y rendimiento.

12. **Iterador no destructivo**

Propón un mecanismo para iterar sobre los elementos del heap en orden decreciente sin alterar su estructura interna ni requerir extracciones. Debes garantizar que la iteración se realice de manera eficiente y que se mantenga la integridad del heap durante el proceso.

15. **Heap persistente (Inmutable)**  

Diseña una versión persistente del DWayHeap, en la que cada modificación (inserción o extracción) genere una nueva versión de la estructura, permitiendo acceder a estados anteriores sin duplicar completamente la información. Analiza la complejidad y el consumo de memoria en este enfoque.

16. **Optimización para operaciones masivas**  

Desarrolla un esquema que optimice el rendimiento del DWayHeap en escenarios de inserciones y extracciones masivas, como en procesamiento de eventos en tiempo real. Investiga técnicas de "bulk-insert" y "batch extraction", justificando el impacto en la complejidad asintótica.

17. **Implementación en un entorno distribuido**  

Propón un diseño que permita distribuir el DWayHeap entre varios nodos de un clúster, asegurando la consistencia y sincronización de las operaciones de inserción y extracción. Evalúa los desafíos en términos de concurrencia, tolerancia a fallos y latencia de red.

18. **Comparación teórica y empírica con otras colas de prioridad**  

Formula un estudio que compare el rendimiento teórico y experimental del DWayHeap frente a otras estructuras de colas de prioridad (por ejemplo, Pairing Heap, Skew Heap o Fibonacci Heap) en diferentes escenarios. Plantea experimentos para medir el impacto del factor de ramificación y analiza en profundidad los resultados obtenidos.


### Respuestas

2. **Fusión de Heaps**

In [ ]:
def merge(heap_a: DWayHeap, heap_b: DWayHeap) -> DWayHeap:
    do:
        # heap_a.insert(heap_b._pairs.pop())
        pair = heap_b._pairs.pop() #Retiro la última hoja
        heap_a.insert(pair) #La inserto como la última hoja y hago operaciones bubble up
    while not heap_b.is_empty()
    return heap_a

Este algoritmo retira las hojas una a una de `heap_b` y las inserta como la última hoja en `heap_a`. Retirar la última hoja (de izquierda a derecha, la que se encuentra más a la derecha es la misma que al final de la lista `_pairs`) mantiene los invariantes de `heap_b`. El método `insert()` se encarga de mantener los invarientes en `heap_a` haciendo operaciones `bubble_up()`.

6. **Construcción en tiempo lineal a partir de un arreglo desordenado**

- Se puede construir un heap rápidamente si, en lugar de armar un heap elemento por elemento, se considera al arreglo desordenado como un árbol d-ario al que solo hay que ordenarle los elementos.
- En el peor caso, los únicos nodos que cumplen con los tres invariantes son los nodos hoja (el arreglo está completamente desordenado). Por lo tanto, solo es necesario ordenarlo a partir del último nodo padre, el cual se encontrará en la posición $\lfloor (|\text{pair} / D|) \rfloor$. Esto reduce enormemente el número de nodos a procesar en el peor caso reduciéndolo en $D^H$, siendo $H$ el número de niveles.
- Para ordenar el árbol hasta un heap, se pueden realizar operaciones `push_down()`. El número de intercambios que se deben realizar desde cualquier nodo de cualquier nivel es de $H-k+1$, siendo $k$ el nivel en el que se encuentra. Para todos los nodos de este nivel, entonces, será $D^k(H-k+1)$.
- Se suman todos estos valores, desde $k = 0$ hasta $k = H-1$
$$ \sum_{k=0}^{H-1}D^k(H-k+1) $$
Haciendo $H-k = m$ se tiene
$$ D^H\sum_{k=0}^{H-1}(m+1)/D^m $$
La sumatoria converge a un término constante y $D^H$ es menor al número de elementos. Entonces, el número total de pasos está acotado por $O(n)$

El algoritmo es el mismo que se usa para `heapify()`:

```
funcion heapify(pares):
    para indice desde (|pares|-1 / D) hasta 0:
        heap.push_down(pares, indice)
    retornar heap
```